In [32]:
import numpy as np
import pandas as pd

pressure = np.repeat([2000, 4000, 6000, 8000, 10000], repeats=3)
density = np.array([
    2.486, 2.479, 2.472, 2.558, 2.570, 2.580,
    2.646, 2.657, 2.653, 2.724, 2.774, 2.808,
    2.861, 2.879, 2.858
])

df = pd.DataFrame({'pressure':pressure, 'density':density})

In [33]:
#  ignore future warnings to keep the output clean

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [34]:
#Fit the OLS regression line model

from statsmodels.formula.api import ols

# Creating the linear model, produce the fit, and print out the summmary of the regression model
model=ols('density~pressure',data=df)
results=model.fit()
print(results.summary2())

                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.981     
Dependent Variable: density          AIC:                -73.0762  
Date:               2024-02-14 17:14 BIC:                -71.6601  
No. Observations:   15               Log-Likelihood:     38.538    
Df Model:           1                F-statistic:        717.1     
Df Residuals:       13               Prob (F-statistic): 9.31e-13  
R-squared:          0.982            Scale:              0.00039636
---------------------------------------------------------------------
              Coef.    Std.Err.      t       P>|t|    [0.025   0.975]
---------------------------------------------------------------------
Intercept     2.3750     0.0121   197.0079   0.0000   2.3490   2.4010
pressure      0.0000     0.0000    26.7780   0.0000   0.0000   0.0001
-------------------------------------------------------------------
Omnibus:                2.101        Durbin-Watson:     

C:\Users\carol\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [35]:
# show the coefficients of the model

results.params

Intercept    2.375000
pressure     0.000049
dtype: float64

In [36]:
print("The regression equation is")
print("density =",f'{results.params["Intercept"]:.4}' ,"+",f'{results.params["pressure"]:.4}',"*pressure")

The regression equation is
density = 2.375 + 4.867e-05 *pressure


In [37]:
# prodcue the ANOVA table

from statsmodels.stats.anova import anova_lm

anova_results = anova_lm(results)
print(anova_results)

            df    sum_sq   mean_sq           F        PR(>F)
pressure   1.0  0.284213  0.284213  717.060422  9.306841e-13
Residual  13.0  0.005153  0.000396         NaN           NaN


In [38]:
# save    model    fits    and    residuals

# Predicting using the model, and obtaining standard errors
pred_summary_frame = results.get_prediction(df["pressure"]).summary_frame(alpha=0.05)[['mean', 'mean_se']]

# Fitted values (i.e., the predicted values of the dependent variable)
ls_fits = pred_summary_frame['mean']

# Mean standard error 
ls_sdfit = pred_summary_frame['mean_se']

# Residuals (i.e., the difference between the observed values and the fitted values)
ls_resids=results.resid

In [39]:
# get the standardized residuals 

#create instance of influence
influence = results.get_influence()

# standardized residuals
ls_standresids = influence.resid_studentized_internal

In [40]:
# display    the    data    along    with    fitted    values,    standard    deviations    of    fit, 
# residuals,    and    standardized    residuals

# Create a DataFrame
data = pd.DataFrame({
    'pressure': pressure,
    'density': density,
    'Fit': ls_fits,
    'StDev Fit': ls_sdfit,  # Assuming std_err is the standard deviation of the fit
    'Residual': ls_resids,
    'St Resid': ls_standresids
})

print(data)

    pressure  density       Fit  StDev Fit  Residual  St Resid
0       2000    2.486  2.472333   0.008903  0.013667  0.767491
1       2000    2.479  2.472333   0.008903  0.006667  0.374386
2       2000    2.472  2.472333   0.008903 -0.000333 -0.018719
3       4000    2.558  2.569667   0.006296 -0.011667 -0.617705
4       4000    2.570  2.569667   0.006296  0.000333  0.017649
5       4000    2.580  2.569667   0.006296  0.010333  0.547110
6       6000    2.646  2.667000   0.005140 -0.021000 -1.091834
7       6000    2.657  2.667000   0.005140 -0.010000 -0.519921
8       6000    2.653  2.667000   0.005140 -0.014000 -0.727889
9       8000    2.724  2.764333   0.006296 -0.040333 -2.135495
10      8000    2.774  2.764333   0.006296  0.009667  0.511813
11      8000    2.808  2.764333   0.006296  0.043667  2.311982
12     10000    2.861  2.861667   0.008903 -0.000667 -0.037439
13     10000    2.879  2.861667   0.008903  0.017333  0.973403
14     10000    2.858  2.861667   0.008903 -0.003667 -0

In [41]:
# or can call some of the fitted results attributes each
results.predict()
results.fittedvalues
results.resid

0     0.013667
1     0.006667
2    -0.000333
3    -0.011667
4     0.000333
5     0.010333
6    -0.021000
7    -0.010000
8    -0.014000
9    -0.040333
10    0.009667
11    0.043667
12   -0.000667
13    0.017333
14   -0.003667
dtype: float64

In [42]:
# or can print the df from the fitted results

results.get_prediction(df["pressure"]).summary_frame(alpha=0.05)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,2.472333,0.008903,2.453099,2.491568,2.425218,2.519449
1,2.472333,0.008903,2.453099,2.491568,2.425218,2.519449
2,2.472333,0.008903,2.453099,2.491568,2.425218,2.519449
3,2.569667,0.006296,2.556066,2.583268,2.524557,2.614776
4,2.569667,0.006296,2.556066,2.583268,2.524557,2.614776
5,2.569667,0.006296,2.556066,2.583268,2.524557,2.614776
6,2.667000,0.005140,2.655895,2.678105,2.622579,2.711421
7,2.667000,0.005140,2.655895,2.678105,2.622579,2.711421
8,2.667000,0.005140,2.655895,2.678105,2.622579,2.711421
9,2.764333,0.006296,2.750732,2.777934,2.719224,2.809443


In [43]:
#get    prediction    for    x    =    5,000    along    with    the    standard    deviation    of    the    fit 
#and    the    95%    confidence    interval

prediction = results.get_prediction(exog=dict(pressure=5000)) 
prediction.summary_frame(alpha=0.05)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,2.618333,0.005452,2.606554,2.630112,2.573739,2.662927
